In [1]:
import xarray as xr 

ds = xr.open_dataset('ml-drought-forecasting/soil-water-forecasting/data/02_intermediate/preprocessed_data.nc')

ds

<xarray.Dataset> Size: 1GB
Dimensions:    (date: 96, latitude: 181, longitude: 360)
Coordinates:
    number     int64 8B ...
  * date       (date) datetime64[ns] 768B 2016-01-01 2016-02-01 ... 2023-12-01
    expver     (date) <U4 2kB ...
  * latitude   (latitude) float64 1kB -90.0 -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
  * longitude  (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/23)
    u10        (date, latitude, longitude) float64 50MB ...
    v10        (date, latitude, longitude) float64 50MB ...
    t2m        (date, latitude, longitude) float64 50MB ...
    sst        (date, latitude, longitude) float64 50MB ...
    sp         (date, latitude, longitude) float64 50MB ...
    tp         (date, latitude, longitude) float64 50MB ...
    ...         ...
    swvl1      (date, latitude, longitude) float64 50MB ...
    cvh        (date, latitude, longitude) float64 50MB ...
    lai_hv     (date, latitude, longitude) float64 50MB ...
    tvh        (date, latitude, longitude) float64 50MB ...
    z          (date, latitude, longitude) float64 50MB ...
    lsm        (date, latitude, longitude) float64 50MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-16T11:34 GRIB to CDM+CF via cfgrib-0.9.1...

Temporal Features

Lets add Seasonal Cycles

In [2]:
# Extract the season from the time coordinate
ds['season'] = ds['date'].dt.season

In [3]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:    (date: 96, latitude: 181, longitude: 360)
Coordinates:
    number     int64 8B ...
  * date       (date) datetime64[ns] 768B 2016-01-01 2016-02-01 ... 2023-12-01
    expver     (date) <U4 2kB ...
  * latitude   (latitude) float64 1kB -90.0 -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
  * longitude  (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/24)
    u10        (date, latitude, longitude) float64 50MB ...
    v10        (date, latitude, longitude) float64 50MB ...
    t2m        (date, latitude, longitude) float64 50MB ...
    sst        (date, latitude, longitude) float64 50MB ...
    sp         (date, latitude, longitude) float64 50MB ...
    tp         (date, latitude, longitude) float64 50MB ...
    ...         ...
    cvh        (date, latitude, longitude) float64 50MB ...
    lai_hv     (date, latitude, longitude) float64 50MB ...
    tvh        (date, latitude, longitude) float64 50MB ...
    z          (date, latitude, longitude) float64 50MB ...
    lsm        (date, latitude, longitude) float64 50MB ...
    season     (date) <U3 1kB 'DJF' 'DJF' 'MAM' 'MAM' ... 'SON' 'SON' 'DJF'
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-16T11:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
# Define a dictionary to map season strings to numerical values
season_mapping = {'DJF': 1, 'MAM': 2, 'JJA': 3, 'SON': 4}

# Convert the DataArray to a pandas Series, map the season names, and convert back to a DataArray
ds['season'] = ds['season'].to_series().map(season_mapping).to_xarray()

In [5]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:    (date: 96, latitude: 181, longitude: 360)
Coordinates:
    number     int64 8B ...
  * date       (date) datetime64[ns] 768B 2016-01-01 2016-02-01 ... 2023-12-01
    expver     (date) <U4 2kB ...
  * latitude   (latitude) float64 1kB -90.0 -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
  * longitude  (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/24)
    u10        (date, latitude, longitude) float64 50MB ...
    v10        (date, latitude, longitude) float64 50MB ...
    t2m        (date, latitude, longitude) float64 50MB ...
    sst        (date, latitude, longitude) float64 50MB ...
    sp         (date, latitude, longitude) float64 50MB ...
    tp         (date, latitude, longitude) float64 50MB ...
    ...         ...
    cvh        (date, latitude, longitude) float64 50MB ...
    lai_hv     (date, latitude, longitude) float64 50MB ...
    tvh        (date, latitude, longitude) float64 50MB ...
    z          (date, latitude, longitude) float64 50MB ...
    lsm        (date, latitude, longitude) float64 50MB ...
    season     (date) int64 768B 1 1 2 2 2 3 3 3 4 4 4 ... 1 2 2 2 3 3 3 4 4 4 1
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-16T11:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
# Broadcast 'season' to match dimensions (date, latitude, longitude)
season_broadcasted = ds['season'].expand_dims({'latitude': ds.latitude, 'longitude': ds.longitude})

# Add this expanded 'season' variable back to the dataset
ds['season'] = season_broadcasted

# Transpose 'season' to match the desired dimensions (date, latitude, longitude)
ds['season'] = ds['season'].transpose('date', 'latitude', 'longitude')

In [7]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:    (date: 96, latitude: 181, longitude: 360)
Coordinates:
    number     int64 8B ...
  * date       (date) datetime64[ns] 768B 2016-01-01 2016-02-01 ... 2023-12-01
    expver     (date) <U4 2kB ...
  * latitude   (latitude) float64 1kB -90.0 -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
  * longitude  (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/24)
    u10        (date, latitude, longitude) float64 50MB ...
    v10        (date, latitude, longitude) float64 50MB ...
    t2m        (date, latitude, longitude) float64 50MB ...
    sst        (date, latitude, longitude) float64 50MB ...
    sp         (date, latitude, longitude) float64 50MB ...
    tp         (date, latitude, longitude) float64 50MB ...
    ...         ...
    cvh        (date, latitude, longitude) float64 50MB ...
    lai_hv     (date, latitude, longitude) float64 50MB ...
    tvh        (date, latitude, longitude) float64 50MB ...
    z          (date, latitude, longitude) float64 50MB ...
    lsm        (date, latitude, longitude) float64 50MB ...
    season     (date, latitude, longitude) int64 50MB 1 1 1 1 1 1 ... 1 1 1 1 1
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-16T11:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
# Convert season to int8
ds['season'] = ds['season'].astype('int32')

In [9]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:    (date: 96, latitude: 181, longitude: 360)
Coordinates:
    number     int64 8B ...
  * date       (date) datetime64[ns] 768B 2016-01-01 2016-02-01 ... 2023-12-01
    expver     (date) <U4 2kB ...
  * latitude   (latitude) float64 1kB -90.0 -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
  * longitude  (longitude) int64 3kB 0 1 2 3 4 5 6 ... 354 355 356 357 358 359
Data variables: (12/24)
    u10        (date, latitude, longitude) float64 50MB ...
    v10        (date, latitude, longitude) float64 50MB ...
    t2m        (date, latitude, longitude) float64 50MB ...
    sst        (date, latitude, longitude) float64 50MB ...
    sp         (date, latitude, longitude) float64 50MB ...
    tp         (date, latitude, longitude) float64 50MB ...
    ...         ...
    cvh        (date, latitude, longitude) float64 50MB ...
    lai_hv     (date, latitude, longitude) float64 50MB ...
    tvh        (date, latitude, longitude) float64 50MB ...
    z          (date, latitude, longitude) float64 50MB ...
    lsm        (date, latitude, longitude) float64 50MB ...
    season     (date, latitude, longitude) int32 25MB 1 1 1 1 1 1 ... 1 1 1 1 1
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-16T11:34 GRIB to CDM+CF via cfgrib-0.9.1...

Soil Moisture Dynamics

Evapotranspiration Balance: Calculate a moisture balance feature as tp - e (precipitation minus evaporation). This balance can serve as an indicator of moisture availability and stress, especially over rolling windows.

Wind Impact: Combine u10 and v10 (wind components) to create a wind speed variable to assess the drying effect of wind on soil moisture.

Determine Wind Direction: Calculate the wind direction using: Wind speed and direction influence evapotranspiration rates, affecting soil moisture levels.

Temperature Anomalies: Identify deviations from long-term average temperatures to capture unusual thermal conditions impacting soil moisture.

In [10]:
ds.to_netcdf(
    'ml-drought-forecasting/soil-water-forecasting/data/04_feature/features.nc'
)